In [ ]:
import numpy as np
from qp.metrics.point_estimate_metric_classes import PointSigmaIQR


In [ ]:
SEED = 1002330
rng = np.random.default_rng(SEED)

estimate = rng.lognormal(mean=1.0, sigma=2, size=1000)
reference = rng.lognormal(mean=1.3, sigma=1.9, size=1000)

In [ ]:
PointSigmaIQR().evaluate(estimate, reference)

In [ ]:
#generator that yields chunks from estimate and reference
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# create an iterator that yields chunks of 100 elements
estimate_chunks = chunker(estimate, 100)
reference_chunks = chunker(reference, 100)

In [ ]:
PointSigmaIQR().eval_from_iterator(estimate_chunks, reference_chunks)
# fails as expected when we get to the `comm` and `centroid` bits in `finalize`.